<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Distilbert_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@start by installing hugging face transfirmer library

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 8.7MB/s 
     |████████████████████████████████| 890kB 14.3MB/s 
     |████████████████████████████████| 3.0MB 14.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7a9d163bc7dc3e72d6080e6a57e847fe2990a26e3ab155737455f95153c4bfbc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#import the libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#use pandas to load the data set
df=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',header=None)

In [ ]:
batch=df[:2000]

In [ ]:
batch[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [ ]:
from transformers import DistilBertModel, DistilBertConfig

SyntaxError: ignored

In [ ]:
model_class=DistilBertModel

In [ ]:
tokenizer_class=DistilBertTokenizer

In [ ]:
pretrained_weights='distilbert-base-uncased'

In [ ]:
#load pretrained mode
tokenizer=tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
model=model_class.from_pretrained(pretrained_weights)

In [ ]:
#model now holds the pretrained  distilbert - smaller, faster and require less memory

Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
tokenized=batch[0].apply(lambda x:tokenizer.encode(x,add_special_tokens=True))

In [ ]:
print(tokenized)

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object


**Padding**

In [ ]:
max_len=0
#find the maximum length of the list
for i in tokenized.values:
  if len(i) > max_len :
    max_len=len(i) 

#concatenate the lists with zeroes for remaining length
padded=np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) 

Print  the shape

In [ ]:
padded.shape

(2000, 59)

Masking

Give attention only to the important items - used attention mask for the same

In [ ]:
attention_mask=np.where(padded!=0,1,0)
attention_mask.shape

(2000, 59)

MODEL #1 and Deep Learning

Now we will use distillbert to  embed all the sentences

In [ ]:
#Construct a tensor with padded attribute
input_ids=torch.tensor(padded)
input_ids


tensor([[  101,  1037, 18385,  ...,     0,     0,     0],
        [  101,  4593,  2128,  ...,     0,     0,     0],
        [  101,  2027,  3653,  ...,     0,     0,     0],
        ...,
        [  101,  2023,  2028,  ...,     0,     0,     0],
        [  101,  1999,  1996,  ...,     0,     0,     0],
        [  101,  1996,  3185,  ...,     0,     0,     0]])

In [ ]:
#convert the attention array to torch tenosr
attention_mask=torch.tensor(attention_mask)

In [ ]:
#Embed the sentence or the input ids
with torch.no_grad():
  last_hidden_states=model(input_ids,attention_mask=attention_mask)
  #The outout is encoded senttence with 768 dim, 
  #the output is rep in 3d way by #number of sentence,#tokens in sentence,sentene embedding dimension
  #here output is (2000,66,768) 

In [ ]:
#print
last_hidden_states[0]

tensor([[[-0.2159, -0.1403,  0.0083,  ..., -0.1369,  0.5867,  0.2011],
         [-0.2471,  0.2468,  0.1008,  ..., -0.1631,  0.9349, -0.0715],
         [ 0.0558,  0.3573,  0.4140,  ..., -0.2430,  0.1770, -0.5080],
         ...,
         [-0.0165,  0.1179,  0.3512,  ..., -0.2401,  0.2722, -0.1750],
         [ 0.0961,  0.0667,  0.3147,  ..., -0.3277,  0.3556, -0.2135],
         [ 0.0454,  0.0519,  0.3168,  ..., -0.2880,  0.1844, -0.1042]],

        [[-0.1726, -0.1448,  0.0022,  ..., -0.1744,  0.2139,  0.3720],
         [ 0.0022,  0.1684,  0.1269,  ..., -0.1888, -0.0195, -0.0283],
         [ 0.0257, -0.2458,  0.0717,  ..., -0.4339,  0.1622,  0.0133],
         ...,
         [ 0.0505, -0.0493,  0.0463,  ..., -0.0448, -0.0540,  0.3136],
         [-0.2128, -0.1907, -0.0215,  ...,  0.0139, -0.2433, -0.0202],
         [-0.1310, -0.1693,  0.1019,  ..., -0.0859, -0.1770, -0.0872]],

        [[-0.0506,  0.0720, -0.0296,  ..., -0.0715,  0.7185,  0.2623],
         [ 0.0536,  0.3136, -0.0598,  ...,  0

In [ ]:
#Access the first token from each of the sentences
features=last_hidden_states[0][:,0,:].numpy()

In [ ]:
features.shape

(2000, 768)

In [ ]:
labels=batch[1]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
#Grid search with best parameters for logistic regression
parameters={'C':np.linspace(0.0001, 100, 20)}
grid_searchCV=GridSearchCV(LogisticRegression(),parameters)
grid_searchCV.fit(train_features,train_labels)

#bestparameters
print('best parameters: ', grid_searchCV.best_params_)
print('best scrores: ', grid_searchCV.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.808


In [ ]:
#perform logistic regression
lr_clf = LogisticRegression(C=5.2)

lr_clf.fit(train_features, train_labels)


LogisticRegression(C=5.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#evalute the score
lr_clf.score(test_features, test_labels)

0.854